# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 27 2022 8:30AM,247318,10,8576746,"Akron BioProducts, LLC",Released
1,Sep 26 2022 3:24PM,247314,19,60017414,"GUSA Granules USA, Inc.",Released
2,Sep 26 2022 3:18PM,247313,10,MSP214301,"Methapharm, Inc.",Released
3,Sep 26 2022 3:18PM,247313,10,MSP214302,"Methapharm, Inc.",Released
4,Sep 26 2022 3:18PM,247313,10,MSP214308,"Methapharm, Inc.",Released
5,Sep 26 2022 3:18PM,247313,10,MSP214309,"Methapharm, Inc.",Released
6,Sep 26 2022 3:18PM,247313,10,MSP214311,"Methapharm, Inc.",Released
7,Sep 26 2022 3:18PM,247313,10,MSP214312,"Methapharm, Inc.",Released
8,Sep 26 2022 3:18PM,247313,10,MSP214313,"Methapharm, Inc.",Released
9,Sep 26 2022 3:18PM,247313,10,MSP214314,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,247311,Released,11
42,247312,Released,3
43,247313,Released,19
44,247314,Released,1
45,247318,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247311,NaN,NaN,11.0
247312,NaN,NaN,3.0
247313,NaN,NaN,19.0
247314,NaN,NaN,1.0
247318,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247220,0.0,2.0,1.0
247223,0.0,0.0,8.0
247238,0.0,0.0,1.0
247240,0.0,1.0,0.0
247246,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247220,0,2,1
247223,0,0,8
247238,0,0,1
247240,0,1,0
247246,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247220,0,2,1
1,247223,0,0,8
2,247238,0,0,1
3,247240,0,1,0
4,247246,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247220,,2,1
1,247223,,,8
2,247238,,,1
3,247240,,1,
4,247246,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 27 2022 8:30AM,247318,10,"Akron BioProducts, LLC"
1,Sep 26 2022 3:24PM,247314,19,"GUSA Granules USA, Inc."
2,Sep 26 2022 3:18PM,247313,10,"Methapharm, Inc."
21,Sep 26 2022 3:16PM,247312,10,"Methapharm, Inc."
24,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation
35,Sep 26 2022 2:52PM,247309,10,"CLINUVEL, Inc."
36,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd"
53,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation
77,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation
80,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 27 2022 8:30AM,247318,10,"Akron BioProducts, LLC",,,1
1,Sep 26 2022 3:24PM,247314,19,"GUSA Granules USA, Inc.",,,1
2,Sep 26 2022 3:18PM,247313,10,"Methapharm, Inc.",,,19
3,Sep 26 2022 3:16PM,247312,10,"Methapharm, Inc.",,,3
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,,,11
5,Sep 26 2022 2:52PM,247309,10,"CLINUVEL, Inc.",,,1
6,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",,,17
7,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,,,24
8,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,,,3
9,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247318,10,"Akron BioProducts, LLC",1,,
1,Sep 26 2022 3:24PM,247314,19,"GUSA Granules USA, Inc.",1,,
2,Sep 26 2022 3:18PM,247313,10,"Methapharm, Inc.",19,,
3,Sep 26 2022 3:16PM,247312,10,"Methapharm, Inc.",3,,
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,,
5,Sep 26 2022 2:52PM,247309,10,"CLINUVEL, Inc.",1,,
6,Sep 26 2022 2:40PM,247308,19,"AdvaGen Pharma, Ltd",17,,
7,Sep 26 2022 2:34PM,247306,10,ISDIN Corporation,24,,
8,Sep 26 2022 2:26PM,247305,10,ISDIN Corporation,3,,
9,Sep 26 2022 2:24PM,247304,10,Yusen – 3D Matrix,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247318,10,"Akron BioProducts, LLC",1,,
1,Sep 26 2022 3:24PM,247314,19,"GUSA Granules USA, Inc.",1,,
2,Sep 26 2022 3:18PM,247313,10,"Methapharm, Inc.",19,,
3,Sep 26 2022 3:16PM,247312,10,"Methapharm, Inc.",3,,
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247318,10,"Akron BioProducts, LLC",1.0,NaN,NaN
1,Sep 26 2022 3:24PM,247314,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Sep 26 2022 3:18PM,247313,10,"Methapharm, Inc.",19.0,NaN,NaN
3,Sep 26 2022 3:16PM,247312,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 27 2022 8:30AM,247318,10,"Akron BioProducts, LLC",1.0,0.0,0.0
1,Sep 26 2022 3:24PM,247314,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Sep 26 2022 3:18PM,247313,10,"Methapharm, Inc.",19.0,0.0,0.0
3,Sep 26 2022 3:16PM,247312,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Sep 26 2022 3:13PM,247311,10,ISDIN Corporation,11.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6676647,115.0,4.0,1.0
16,741812,5.0,0.0,0.0
19,1236456,32.0,12.0,0.0
20,741855,33.0,18.0,11.0
21,247246,0.0,1.0,0.0
22,247298,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6676647,115.0,4.0,1.0
1,16,741812,5.0,0.0,0.0
2,19,1236456,32.0,12.0,0.0
3,20,741855,33.0,18.0,11.0
4,21,247246,0.0,1.0,0.0
5,22,247298,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,115.0,4.0,1.0
1,16,5.0,0.0,0.0
2,19,32.0,12.0,0.0
3,20,33.0,18.0,11.0
4,21,0.0,1.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,115.0
1,16,Released,5.0
2,19,Released,32.0
3,20,Released,33.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,11.0,0.0,0.0
Executing,4.0,0.0,12.0,18.0,1.0,0.0
Released,115.0,5.0,32.0,33.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20,21,22
0,Completed,1.0,0.0,0.0,11.0,0.0,0.0
1,Executing,4.0,0.0,12.0,18.0,1.0,0.0
2,Released,115.0,5.0,32.0,33.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20,21,22
0,Completed,1.0,0.0,0.0,11.0,0.0,0.0
1,Executing,4.0,0.0,12.0,18.0,1.0,0.0
2,Released,115.0,5.0,32.0,33.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()